In [1]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv('../data/train.csv')

Convert data to lowercase and remove all URLs

In [2]:
cleanedData = data['text'].str.lower()
cleanedData = cleanedData.replace(to_replace = r'https?:\/\/.*[\r\n]*|[^\w\s]', value = " ", regex=True)

Tokenize each string

In [3]:
tokens = []

for i in range(len(cleanedData)):
    tokens.append([data for data in cleanedData[i].split(" ") if data])

Find frequency of each token

In [4]:
frequencies = {} # token : frequency
for row in tokens:
    for word in row:
        frequencies[word] = frequencies.get(word, 0) + 1

Find rank of each token. Low rank (ie. 1) means word is less frequent

In [22]:
frequency_set = sorted(set(list(frequencies.values())))
num_ranks = len(frequency_set)

frequency_to_rank = {}

for i, freq in enumerate(frequency_set):
    frequency_to_rank[freq] = i+1

ranks = {} # token : rank

for key, value in frequencies.items():
    ranks[key] = frequency_to_rank[value]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 102, 103, 104, 108, 109, 110, 111, 113, 114, 115, 117, 119, 120, 125, 126, 127, 128, 129, 130, 132, 136, 139, 141, 146, 152, 154, 156, 164, 167, 174, 183, 184, 187, 188, 189, 193, 199, 202, 210, 215, 219, 220, 224, 229, 238, 239, 241, 245, 248, 250, 251, 256, 257, 261, 272, 286, 287, 289, 295, 299, 315, 317, 319, 321, 325, 326, 327, 343, 385, 400, 402, 416, 474, 480, 508, 535, 566, 615, 674, 763, 843, 875, 891, 892, 935, 1376, 1754, 1800, 1921, 1952, 2160, 3222]
187
{'our': 93, 'deeds': 2, 'are': 166, 'the': 187, 'reason': 19, 'of': 183, 'this': 168, 'earthquake': 41, 'may': 82, 'allah': 9, 'forgive': 2, 'us': 103, 'all'

In [23]:
test_case_index = 33

print("1. Example entry:\n\t", data['text'][test_case_index])
print("\n2. Cleaned entry:\n\t",cleanedData[test_case_index])
print("\n3. Tokenize:\n\t", tokens[test_case_index])

print("\n4. Token Frequencies and Ranks:\n\t[token] : [frequency] | [rank]")
for token in tokens[test_case_index]:
    print(f"\t{token} : {frequencies[token]} | {ranks[token]}")

1. Example entry:
	 #AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi

2. Cleaned entry:
	  africanbaze  breaking news nigeria flag set ablaze in aba   

3. Tokenize:
	 ['africanbaze', 'breaking', 'news', 'nigeria', 'flag', 'set', 'ablaze', 'in', 'aba']

4. Token Frequencies and Ranks:
	[token] : [frequency] | [rank]
	africanbaze : 1 | 1
	breaking : 40 | 40
	news : 154 | 121
	nigeria : 4 | 4
	flag : 21 | 21
	set : 47 | 47
	ablaze : 28 | 28
	in : 1952 | 185
	aba : 14 | 14
